In [1]:
import os
import pandas as pd

from pandas import DataFrame
from datetime import datetime, timezone, timedelta
from io import StringIO
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, request_influxdb, query_tangaras, query_measure, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv")

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 12


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.378983,-76.541061
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
4,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-09-26T00:00:00-05:00 -> 1695704400000, Until: 2023-09-26T00:05:00-05:00 -> 1695704700000


In [4]:
def df_temp_sensors(df_tangaras: DataFrame, start_timestamp: int, end_timestamp: int) -> DataFrame:
    """
    Get Temperature Data Frame Sensors of all Tangara sensors that have reported data over a period of time.
    
    :params:
    :df_tangaras: DataFrame, Tangaras DataFrame
    :start_timestamp: int, timestamp datetime value, ms
    :end_timestamp: int, timestamp datetime value, ms

    :return: DataFrame, Temperature Data Frame Sensors
    """
    # Data Frame Sensors List
    df_sensors_list: [DataFrame] = []
    # SQL Query Data Sensors
    influxdb_sql_query_measure = query_measure(df_tangaras['MAC'].to_list(), start_timestamp, end_timestamp, datatype='tmp')
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(influxdb_sql_query_measure)
    #print(influxdb_request)
    #print(influxdb_request.text)

    # Data Frame InfluxDB Sensors
    df_influxdb_sensors = pd.read_csv(StringIO(influxdb_request.text), sep=",", low_memory=False)

    # Remove/Add Columns
    df_influxdb_sensors = df_influxdb_sensors[['time', 'name.1', 'last']]
    df_influxdb_sensors.rename(columns={'time': 'DATETIME', 'name.1': 'MAC', 'last': 'TEMP'}, inplace=True)

    # Truncate Response
    for _, row in df_tangaras.iterrows():
        df_sensor = df_influxdb_sensors.loc[df_influxdb_sensors['MAC'] == row['MAC']].reset_index(drop=True)[['DATETIME', 'TEMP']] # Warning
        if not df_sensor.empty:
            df_sensor.rename(columns={'TEMP': row['ID']}, inplace=True)
            df_sensor.set_index('DATETIME', inplace=True)
            df_sensors_list.append(df_sensor)
    
    df_sensors = df_sensors_list[0].join(df_sensors_list[1:]).reset_index()

    # Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
    tz = timezone(timedelta(hours=-5))
    df_sensors['DATETIME'] = df_sensors['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz).isoformat())
    df_sensors['DATETIME'] = pd.to_datetime(df_sensors['DATETIME'])

    # Set Index
    df_sensors.set_index('DATETIME', inplace=True)
    df_sensors = df_sensors.asfreq(freq='30S')

    df_sensors[df_sensors.columns.to_list()] = df_sensors[df_sensors.columns.to_list()].astype('float64')
    
    return df_sensors

In [5]:
# Temperature Data Frame Sensors
df_temp_raw = df_temp_sensors(df_tangaras, start_timestamp, end_timestamp)
df_temp_raw.head()
#df_temp_raw.shape

,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2E9A,TANGARA_2FF6,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,
2023-09-26 00:00:30-05:00,0.0,30.17,28.22,29.50,29.60,30.18,28.98,26.72,28.9,27.3,31.09,29.26
2023-09-26 00:01:00-05:00,0.0,30.20,28.25,29.51,29.65,30.18,28.89,26.70,28.9,27.3,31.09,29.30
2023-09-26 00:01:30-05:00,0.0,30.20,28.23,29.49,29.62,30.17,28.88,26.70,28.9,NaN,31.10,29.32
2023-09-26 00:02:00-05:00,0.0,30.20,28.26,29.49,29.65,30.14,28.83,26.69,28.9,27.3,31.08,29.36
2023-09-26 00:02:30-05:00,0.0,30.18,28.25,29.46,29.65,30.14,28.79,26.67,28.9,27.3,31.12,29.37


## Descriptive Statistics

In [6]:
# Describe Data
df_temp_raw.describe()

,TANGARA_25CE,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2E9A,TANGARA_2FF6,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000e+00,8.000000,9.000000,9.000000
mean,0.0,30.183333,28.246667,29.464444,29.657778,30.122222,28.796667,26.685556,2.890000e+01,27.362500,31.096667,29.355556
std,0.0,0.017321,0.013229,0.033208,0.033458,0.065532,0.106888,0.021279,3.768222e-15,0.074402,0.011180,0.054339
min,0.0,30.160000,28.220000,29.430000,29.600000,29.970000,28.670000,26.660000,2.890000e+01,27.300000,31.080000,29.260000
25%,0.0,30.170000,28.250000,29.430000,29.650000,30.100000,28.690000,26.670000,2.890000e+01,27.300000,31.090000,29.320000
50%,0.0,30.180000,28.250000,29.460000,29.650000,30.140000,28.790000,26.690000,2.890000e+01,27.350000,31.100000,29.370000
75%,0.0,30.200000,28.250000,29.490000,29.690000,30.170000,28.880000,26.700000,2.890000e+01,27.400000,31.100000,29.380000
max,0.0,30.200000,28.260000,29.510000,29.700000,30.180000,28.980000,26.720000,2.890000e+01,27.500000,31.120000,29.440000


## Missing Data

In [7]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_532E, Data: 89%, Missing: 11%, To be checked


## Data Correlation

In [25]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
if not is_ok:
    print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: -0.94, To be checked


In [27]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")